# Montado de Mi unidad de Drive para acceso y gestión de ficheros (Google Colab)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Conversión de formato log a JSON válido

In [ ]:
import json
import os

def obtener_entradas(text):
    stack = []
    objs = []
    inicio = None

    for i, char in enumerate(text):
        if char == '{':
            if not stack:
                inicio = i
            stack.append('{')
        elif char == '}':
            if stack:
                stack.pop()
                if not stack:
                    obj_str = text[inicio:i+1]
                    try:
                        obj = json.loads(obj_str)
                        objs.append(obj)
                    except json.JSONDecodeError:
                        pass
    return objs

def convertir_log_a_json(input_file, output_file):
    with open(input_file, 'r', encoding='utf-8') as f:
        contenido = f.read()

    json_objects = obtener_entradas(contenido)

    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(json_objects, f, indent=2, ensure_ascii=False)

    print(f"JSON válido generado en {output_file}")

# Directorios
base_dir_1 = '/content/drive/MyDrive/datos-tabletas/tableta-1'
base_dir_2 = '/content/drive/MyDrive/datos-tabletas/tableta-2'

# Función para procesar el directorio
def procesar_tableta(base_dir):
    log_dir = os.path.join(base_dir, 'files')
    json_dir = os.path.join(base_dir, 'json')

    for filename in os.listdir(log_dir):
        if filename.endswith('.log'):
            input_path = os.path.join(log_dir, filename)
            output_filename = os.path.splitext(filename)[0] + '.json'
            output_path = os.path.join(json_dir, output_filename)
            convertir_log_a_json(input_path, output_path)

# Procesa los archivos de ambos directorios
procesar_tableta(base_dir_1)
procesar_tableta(base_dir_2)

# Obtención de resultados de tabletas



In [ ]:
# Ruta de la tableta 1
folder_paths = ["/content/drive/MyDrive/datos-tabletas/tableta-1/json", "/content/drive/MyDrive/datos-tabletas/tableta-2/json"]

##Datos de usuarios

### Perfiles de usuarios

In [ ]:
import json
import os

# Diccionario para almacenar la información de los usuarios
usuarios = {}

# Recorremos todos los archivos JSON en la carpeta
for folder_path in folder_paths:
  for filename in os.listdir(folder_path):
          path = os.path.join(folder_path, filename)

          # Abrimos el archivo JSON
          with open(path, 'r') as f:
              data = json.load(f)

              # Buscamos las entradas con objeto "player"
              for entry in data:
                  if 'player' in entry:
                      player = entry['player']
                      user_id = player.get('userId')

                      # Si ya existe el userId, se actualiza la información
                      usuarios[user_id] = {
                          "userName": player.get('userName'),
                          "userAge": player.get('userAge'),
                          "userGender": player.get('userGender'),
                          "userLevel": player.get('userLevel'),
                      }

# Número de participantes
print(f"\nSe encontraron {len(usuarios)} participantes únicos\n")
print("-" * 30)

# Información de los usuarios obtenida
for user_id, user_info in usuarios.items():
    print(f"Id: {user_id}")
    print(f"Nombre: {user_info['userName']}")
    print(f"Edad: {user_info['userAge']}")
    print(f"Género: {user_info['userGender']}")
    print(f"Grado de TEA: {user_info['userLevel']}")
    print("-" * 30)

### Análisis demográfico

#### Distribución por edad

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Convertir el diccionario a DataFrame
usuarios_df = pd.DataFrame.from_dict(usuarios, orient='index')

# Distribución por edades
plt.figure(figsize=(10, 6))
age_count = usuarios_df['userAge'].value_counts().sort_index()
plt.bar(age_count.index, age_count.values, color='lightblue')

plt.title('Distribución de edades de participantes')
plt.xlabel('Edad')
plt.ylabel('Número de participantes')
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()

#### Distribución por género

In [ ]:
plt.figure(figsize=(10, 6))

gender_count = usuarios_df['userGender'].value_counts()
label = {'M': 'Masculino', 'F': 'Femenino'}
gender_count.index = gender_count.index.map(lambda x: label.get(x, x))

plt.pie(
    gender_count,
    labels=gender_count.index,
    autopct='%1.0f%%'
)

plt.title('Distribución por género')
plt.tight_layout()
plt.show()

#### Distribución por Grado de TEA

In [ ]:
plt.figure(figsize=(10, 6))

usuarios_df['userLevel'].value_counts().plot(
    kind='bar'
)

plt.title('Distribución por Grado de TEA')
plt.xlabel('Grado')
plt.ylabel('Número de Participantes')
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()

## Datos de juego



### Tiempo de juego de cada uno, en 1a y 2a sesión.


In [ ]:
from datetime import datetime, timedelta

# Duración de juego de cada usuario por día
jugadores_duracion_dia = {}

# Duración total de cada usuario
jugadores_duracion_total = {}
tiempo_total = timedelta()

# Recorremos todos los archivos JSON en la carpeta
for folder_path in folder_paths:
  for filename in os.listdir(folder_path):

          # Obtener la fecha del nombre del archivo
          fecha_juego = filename.split('.')[0]
          fecha_juego = datetime.strptime(fecha_juego, "%d-%m-%Y")

          path = os.path.join(folder_path, filename)

          # Abrimos el archivo JSON
          with open(path, 'r') as f:
              data = json.load(f)

              # Recorremos el archivo JSON
              for entry in data:

                  # Buscamos entradas con el objeto 'GAME'
                  if entry.get("object") == "GAME":

                      player_id = entry.get("playerId")
                      timestamp = entry.get("timestamp")

                      # Convertimos el timestamp a un datetime para comparaciones
                      timestamp = datetime.fromisoformat(timestamp[:-1])

                      # Calculamos la duración de la sesión ese día
                      if player_id not in jugadores_duracion_dia:
                          jugadores_duracion_dia[player_id] = {}

                      if fecha_juego not in jugadores_duracion_dia[player_id]:
                          jugadores_duracion_dia[player_id][fecha_juego] = {
                              "primer_juego": timestamp,
                              "ultimo_juego": timestamp
                          }
                      else:
                          # Actualizamos la fecha del último juego ese día
                          if timestamp > jugadores_duracion_dia[player_id][fecha_juego]["ultimo_juego"]:
                              jugadores_duracion_dia[player_id][fecha_juego]["ultimo_juego"] = timestamp


minutos_jugadores = []

# Mostramos la duración de sesión de cada jugador por día y la duración total
for player_id, fechas in jugadores_duracion_dia.items():
    duracion_total = timedelta(0)
    print(f"Jugador {player_id}:")

    for fecha, tiempos in fechas.items():
        duracion_dia = tiempos["ultimo_juego"] - tiempos["primer_juego"]
        duracion_total += duracion_dia

        # Extraemos minutos de la duración
        minutos_dia = duracion_dia.total_seconds() / 60

        print(f"  En el día {fecha.strftime('%d-%m-%Y')}, jugó durante {minutos_dia:.1f} minutos.")

    # Calculamos la duración total
    minutos_total = duracion_total.total_seconds() / 60

    tiempo_total += duracion_total
    minutos_jugadores.append(minutos_total)

    print(f"  Duración total de juego: {minutos_total:.1f} minutos.\n")

promedio = tiempo_total.total_seconds() / (60 * len(usuarios))

print("-" * 30)
print(f"Promedio de tiempo jugado entre todos los jugadores: {promedio:.1f} minutos.")



In [ ]:
import seaborn as sns

# Convertimos a DataFrame
df = pd.DataFrame(minutos_jugadores, columns=['tiempo_minutos'])

# Calculamos estadísticas
media = df['tiempo_minutos'].mean()
mediana = df['tiempo_minutos'].median()
min_val = df['tiempo_minutos'].min()
max_val = df['tiempo_minutos'].max()
q1 = df['tiempo_minutos'].quantile(0.25)
q3 = df['tiempo_minutos'].quantile(0.75)

# Creamos la figura
plt.figure(figsize=(10, 6))
ax = sns.boxplot(x='tiempo_minutos', data=df,
                width=0.5,
                orient='h',
                medianprops=dict(color="red", linewidth=2.5))  # Mediana

# Añadimos puntos para visualizar los datos individuales
sns.stripplot(x='tiempo_minutos', data=df,
              size=7, color='0.3', alpha=0.6,
              orient='h')

plt.title("Distribución del tiempo jugado por jugador", fontsize=16)
plt.xlabel("Tiempo jugado (minutos)", fontsize=14)
plt.ylabel("")
plt.grid(axis='x', linestyle='--', alpha=0.7)

# Añadimos línea vertical para la media
plt.axvline(x=media, color='green', linestyle='--', linewidth=2)

# Añadimos estadísticas en un recuadro
stats = (
    f"Tamaño muestra: {len(df)}\n"
    f"Media: {media:.1f} min\n"
    f"Mediana: {mediana:.1f} min\n"
    f"Mínimo: {min_val:.1f} min\n"
    f"Máximo: {max_val:.1f} min\n"
)

# Posicionamos el texto en la esquina superior derecha
plt.annotate(stats, xy=(0.98, 0.95), xycoords='axes fraction',
            bbox=dict(boxstyle="round,pad=0.5", facecolor="white", alpha=0.8),
            va='top', ha='right', fontsize=10)

plt.tight_layout()
plt.show()

### Juegos jugados en cada sesión: cuál es más popular? cómo se suelen organizar los minijuegos de cada sesión? Qué partes de qué juegos son más fáciles / difíciles, medible tanto por aciertos vs fallos como por tiempo-en-acabar.

In [ ]:
#Composición de las partidas de los usuarios
partidas_usuarios = {}
contador_minijuegos = {}

for folder_path in folder_paths:
  for filename in os.listdir(folder_path):

          fecha_juego = filename.split('.')[0]
          fecha_juego = datetime.strptime(fecha_juego, "%d-%m-%Y")

          path = os.path.join(folder_path, filename)

          with open(path, 'r') as f:
              data = json.load(f)

              bloque_actual = None

              for entry in data:
                  if entry.get("object") == "GAME" and entry.get("action") == "INITIALIZED":
                      bloque_actual = {
                          "playerId": entry.get("playerId"),
                          "gameType": entry.get("gameType"),
                          "correctas": 0,
                          "incorrectas": 0,
                          "category": entry.get("category"),
                          "imagesPerRound": entry.get("imagesPerRound"),
                          "rounds": entry.get("rounds"),
                          "emotion": entry.get("category"),
                          "sequence": entry.get("category"),
                          "imagesCorrectsPerRound": entry.get("imagesCorrectsPerRound")
                      }

                  elif bloque_actual and entry.get("action") == "SELECTED" and entry.get("object") == "ROUND":
                      result = entry.get("result")
                      if result is True:
                          bloque_actual["correctas"] += 1
                      elif result is False:
                          bloque_actual["incorrectas"] += 1

                  elif bloque_actual and entry.get("object") == "GAME" and entry.get("action") == "COMPLETED":
                      player_id = bloque_actual["playerId"]
                      if player_id not in partidas_usuarios:
                          partidas_usuarios[player_id] = {}
                      if fecha_juego not in partidas_usuarios[player_id]:
                          partidas_usuarios[player_id][fecha_juego] = []

                      game_type = bloque_actual["gameType"]
                      contador_minijuegos[game_type] = contador_minijuegos.get(game_type, 0) + 1

                      partidas_usuarios[player_id][fecha_juego].append(bloque_actual)
                      bloque_actual = None

# Mostrar resultados
for player_id, fechas in partidas_usuarios.items():
    print(f"Jugador {player_id}:")

    for fecha, partidas in fechas.items():
        print(f"  En el día {fecha.strftime('%d-%m-%Y')} jugó los siguientes minijuegos en orden:")

        for i, partida in enumerate(partidas, 1):
            game_type = partida["gameType"]
            texto_adicional = f"Aciertos: {partida['correctas']}, Errores: {partida['incorrectas']}"

            if game_type == "VOCABULARY":
                category = partida.get("category")
                images_per_round = partida.get("imagesPerRound")
                rounds = partida.get("rounds")
                texto_adicional += f", Categoría: {category}, Imágenes por ronda: {images_per_round}, Rondas: {rounds}"

            elif game_type == "SEQUENCE":
                sequence = partida.get("sequence")
                if sequence:
                    texto_adicional += f", Secuencia: {sequence}"

            elif game_type == "EMOTIONS":
                emotion = partida.get("emotion")
                images_per_round = partida.get("imagesPerRound")
                corrects_per_round = partida.get("imagesCorrectsPerRound")
                rounds = partida.get("rounds")
                texto_adicional += f", Emoción: {emotion}, Imágenes por ronda: {images_per_round}, Respuestas correctas por ronda: {corrects_per_round}, Rondas: {rounds}"

            print(f"    {game_type} - {texto_adicional}")

print("-" * 30)
print("Total de partidas por tipo de minijuego:")
for game_type, total in contador_minijuegos.items():
    print(f"  {game_type}: {total} partidas")

#### Aciertos vs. fallos por tipo de juego

In [ ]:
# Contadores por tipo de juego
estadisticas_por_juego = {}
datos = []

for jugador, fechas in partidas_usuarios.items():
    for partidas in fechas.values():
        for partida in partidas:
            tipo = partida["gameType"]
            correctas = partida["correctas"]
            incorrectas = partida["incorrectas"]
            total = correctas + incorrectas

            if tipo not in estadisticas_por_juego:
                estadisticas_por_juego[tipo] = {"correctas": 0, "incorrectas": 0}

            estadisticas_por_juego[tipo]["correctas"] += correctas
            estadisticas_por_juego[tipo]["incorrectas"] += incorrectas

            error_pct = (incorrectas / total) * 100
            datos.append({
                    'tipo_juego': tipo,
                    'tasa_error_porcentaje': error_pct
                })

# Convertimos a DataFrame
df = pd.DataFrame(estadisticas_por_juego).T
df["total"] = df["correctas"] + df["incorrectas"]
df["acierto_%"] = (df["correctas"] / df["total"]) * 100
df["fallo_%"] = (df["incorrectas"] / df["total"]) * 100

# Aciertos vs fallos por tipo de juego
ax = df[["correctas", "incorrectas"]].plot(
    kind="bar",
    stacked=True,
    figsize=(10, 6),
    color=["green", "red"]
)

plt.title("Aciertos vs fallos por minijuego")
plt.ylabel("Número de respuestas")
plt.xlabel("Minijuego")
plt.xticks(rotation=0)
plt.legend(["Aciertos", "Fallos"])
plt.tight_layout()

# Añadir porcentajes sobre las barras
for idx, row in enumerate(df.itertuples()):
    total = row.total
    aciertos_pct = f"{df.iloc[idx]['acierto_%']:.0f}%"
    fallos_pct = f"{df.iloc[idx]['fallo_%']:.0f}%"

    # Posición de aciertos
    ax.text(idx, row.correctas / 2, aciertos_pct, ha='center', va='center', color='white', fontsize=10)

    # Posición de fallos
    ax.text(idx, row.correctas + (row.incorrectas / 2), fallos_pct, ha='center', va='center', color='white', fontsize=10)

plt.show()

In [ ]:
# Crear DataFrame
df_tasas = pd.DataFrame(datos)
tipos_juego = df_tasas['tipo_juego'].unique()

# Crear el box plot
plt.figure(figsize=(10, 6))

ax = sns.boxplot(x='tipo_juego', y='tasa_error_porcentaje', data=df_tasas,
                    medianprops=dict(color="red", linewidth=2.5),
                    width=0.5)

# Añadimos puntos para visualizar los datos individuales
sns.stripplot(x='tipo_juego', y='tasa_error_porcentaje', data=df_tasas,
                size=4, color='.3', alpha=0.5)

    # Personalizamos el gráfico
plt.title("Distribución de tasas de error por minijuego", fontsize=16)
plt.xlabel("Minijuego")
plt.ylabel("Tasa de error (%)")
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Calcular estadísticas por minijuego
estadisticas = df_tasas.groupby('tipo_juego')['tasa_error_porcentaje'].agg(['mean', 'median', 'min', 'max', 'count'])

# Añadir las medias como puntos en el boxplot
for i, tipo in enumerate(tipos_juego):
    if tipo in estadisticas.index:
            media = estadisticas.loc[tipo, 'mean']
            plt.plot(i, media, marker='D', color='red', markersize=8,
                    label='Media' if i==0 else "")

    # Añadimos anotaciones con estadísticas por tipo
    y_max = df_tasas['tasa_error_porcentaje'].max() * 1.6
    plt.ylim(-5, y_max)

    for i, tipo in enumerate(tipos_juego):
        if tipo in estadisticas.index:
            stats = estadisticas.loc[tipo]
            stats_text = (
                f"Tamaño muestra: {int(stats['count'])}\n"
                f"Media: {stats['mean']:.1f}%\n"
                f"Mediana: {stats['median']:.1f}%\n"
                f"Mín: {stats['min']:.1f}%\n"
                f"Máx: {stats['max']:.1f}%"
            )

            # Posición de la anotación
            plt.annotate(stats_text, xy=(i, y_max * 0.9), ha='center', va='top',
                       bbox=dict(boxstyle="round,pad=0.5", facecolor="white", alpha=0.7),
                       fontsize=9)


#### Evolución tasas de errores por grado de TEA

In [ ]:
import seaborn as sns

# Tasas de errores por partida
tasa_errores_por_partida = []

for jugador, fechas in partidas_usuarios.items():
    for fechas_partidas in fechas.values():
        for i, partida in enumerate(fechas_partidas):
            tipo = partida["gameType"]
            correctas = partida["correctas"]
            incorrectas = partida["incorrectas"]
            total_respuestas = correctas + incorrectas

            tasa_errores = incorrectas / total_respuestas

            tasa_errores_por_partida.append({
                "jugador": jugador,
                "fecha": fechas_partidas[0].get('fecha_juego', 'Desconocida'),  # Si tienes la fecha
                "partida": i + 1,
                "tipo_de_juego": tipo,
                "tasa_error": tasa_errores
            })

# Añadir grado de TEA a cada entrada
for entrada in tasa_errores_por_partida:
    jugador = entrada['jugador']
    entrada['userLevel'] = usuarios.get(jugador, {}).get('userLevel')

# Creamos un DataFrame
df = pd.DataFrame(tasa_errores_por_partida)
df['tasa_error_porcentaje'] = df['tasa_error'] * 100
order = ['G1', 'G2', 'G3']

# Calcular estadísticas por grado de TEA
estadisticas = df.groupby('userLevel')['tasa_error_porcentaje'].agg(['count', 'mean', 'median', 'min', 'max'])

# Crear el boxplot
plt.figure(figsize=(10, 6))
sns.boxplot(x='userLevel', y='tasa_error_porcentaje', data=df, order=order, medianprops=dict(color="red", linewidth=2.5))
sns.stripplot(x='userLevel', y='tasa_error_porcentaje', data=df, size=4, color='.3', alpha=0.6, order=order)

plt.title("Distribución de tasas de errores por grado de TEA", fontsize=14)
plt.xlabel("Grado de TEA")
plt.ylabel("Tasa de errores (%)")
plt.ylim(0, 100)
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Añadir la media como punto
for i, nivel in enumerate(order):
    if nivel in estadisticas.index:
        plt.plot(i, estadisticas.loc[nivel, 'mean'], marker='D', color='red', markersize=8)

# Añadir anotaciones
y_max = df['tasa_error_porcentaje'].max() * 1.4
plt.ylim(-5, y_max)

for i, nivel in enumerate(order):
    if nivel in estadisticas.index:
        stats = estadisticas.loc[nivel]
        texto = (
            f"Muestra: {int(stats['count'])}\n"
            f"Media: {stats['mean']:.1f}%\n"
            f"Mediana: {stats['median']:.1f}%\n"
            f"Mín: {stats['min']:.1f}%\n"
            f"Máx: {stats['max']:.1f}%"
        )
        plt.annotate(
            texto,
            xy=(i, y_max * 0.9),
            ha='center',
            va='top',
            fontsize=9,
            bbox=dict(boxstyle="round,pad=0.4", facecolor="white", alpha=0.8)
        )

plt.tight_layout()
plt.show()

In [ ]:
# Filtrar por un tipo de juego específico
df_filtrado = df[df['tipo_de_juego'] == 'VOCABULARY']

# Estadísticas por grado de TEA
estadisticas = df_filtrado.groupby('userLevel')['tasa_error_porcentaje'].agg(['count', 'mean', 'median', 'min', 'max'])
orden_grados = ['G1', 'G2', 'G3']

# Crear el box plot solo con ese tipo de juego
plt.figure(figsize=(10, 6))
sns.boxplot(x='userLevel', y='tasa_error_porcentaje', data=df_filtrado, order=order, medianprops=dict(color="red", linewidth=2.5))
sns.stripplot(x='userLevel', y='tasa_error_porcentaje', data=df_filtrado, size=4, color='.3', alpha=0.6, order=order)

plt.title("Tasa de errores en el minijuego 'Vocabulario' por grado de TEA", fontsize=14)
plt.xlabel("Grado de TEA")
plt.ylabel("Tasa de errores (%)")
plt.ylim(0, 100)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()

# Calcular y graficar medias
medias = df_filtrado.groupby('userLevel')['tasa_error_porcentaje'].mean()
for i, nivel in enumerate(order):
    if nivel in medias:
        plt.plot(i, medias[nivel], marker='D', color='red', markersize=6)

# Anotaciones estadísticas por grado
y_max = df_filtrado['tasa_error_porcentaje'].max() * 1.4
plt.ylim(-5, y_max)

for i, grado in enumerate(orden_grados):
    if grado in estadisticas.index:
        stats = estadisticas.loc[grado]
        texto = (
            f"Muestra: {int(stats['count'])}\n"
            f"Media: {stats['mean']:.1f}%\n"
            f"Mediana: {stats['median']:.1f}%\n"
            f"Mín: {stats['min']:.1f}%\n"
            f"Máx: {stats['max']:.1f}%"
        )
        plt.annotate(
            texto,
            xy=(i, y_max * 0.9),
            ha='center',
            va='top',
            fontsize=9,
            bbox=dict(boxstyle="round,pad=0.4", facecolor="white", alpha=0.8)
        )

plt.show()


In [ ]:
# Filtrar por un tipo de juego específico
df_filtrado = df[df['tipo_de_juego'] == 'EMOTIONS']

# Estadísticas por grado de TEA
estadisticas = df_filtrado.groupby('userLevel')['tasa_error_porcentaje'].agg(['count', 'mean', 'median', 'min', 'max'])
orden_grados = ['G1', 'G2', 'G3']

# Crear el box plot solo con ese tipo de juego
plt.figure(figsize=(10, 6))
sns.boxplot(x='userLevel', y='tasa_error_porcentaje', data=df_filtrado, order=order, medianprops=dict(color="red", linewidth=2.5))
sns.stripplot(x='userLevel', y='tasa_error_porcentaje', data=df_filtrado, size=4, color='.3', alpha=0.6, order=order)

plt.title("Tasa de errores en el minijuego 'Emociones' por grado de TEA", fontsize=14)
plt.xlabel("Grado de TEA")
plt.ylabel("Tasa de errores (%)")
plt.ylim(0, 100)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()

# Calcular y graficar medias
medias = df_filtrado.groupby('userLevel')['tasa_error_porcentaje'].mean()
for i, nivel in enumerate(order):
    if nivel in medias:
        plt.plot(i, medias[nivel], marker='D', color='red', markersize=6)

# Anotaciones estadísticas por grado
y_max = df_filtrado['tasa_error_porcentaje'].max() * 1.6
plt.ylim(-5, y_max)

for i, grado in enumerate(orden_grados):
    if grado in estadisticas.index:
        stats = estadisticas.loc[grado]
        texto = (
            f"Muestra: {int(stats['count'])}\n"
            f"Media: {stats['mean']:.1f}%\n"
            f"Mediana: {stats['median']:.1f}%\n"
            f"Mín: {stats['min']:.1f}%\n"
            f"Máx: {stats['max']:.1f}%"
        )
        plt.annotate(
            texto,
            xy=(i, y_max * 0.9),
            ha='center',
            va='top',
            fontsize=9,
            bbox=dict(boxstyle="round,pad=0.4", facecolor="white", alpha=0.8)
        )

plt.show()


In [ ]:
# Filtrar por un tipo de juego específico
df_filtrado = df[df['tipo_de_juego'] == 'SEQUENCE']

# Estadísticas por grado de TEA
estadisticas = df_filtrado.groupby('userLevel')['tasa_error_porcentaje'].agg(['count', 'mean', 'median', 'min', 'max'])
orden_grados = ['G1', 'G2', 'G3']

# Crear el box plot solo con ese tipo de juego
plt.figure(figsize=(10, 6))
sns.boxplot(x='userLevel', y='tasa_error_porcentaje', data=df_filtrado, order=order, medianprops=dict(color="red", linewidth=2.5))
sns.stripplot(x='userLevel', y='tasa_error_porcentaje', data=df_filtrado, size=4, color='.3', alpha=0.6, order=order)

plt.title("Tasa de errores en el minijuego 'Secuencia' por grado de TEA", fontsize=14)
plt.xlabel("Grado de TEA")
plt.ylabel("Tasa de errores (%)")
plt.ylim(0, 100)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()

# Calcular y graficar medias
medias = df_filtrado.groupby('userLevel')['tasa_error_porcentaje'].mean()
for i, nivel in enumerate(order):
    if nivel in medias:
        plt.plot(i, medias[nivel], marker='o', color='red', markersize=6)

# Anotaciones estadísticas por grado
y_max = df_filtrado['tasa_error_porcentaje'].max() * 1.7
plt.ylim(-5, y_max)

for i, grado in enumerate(orden_grados):
    if grado in estadisticas.index:
        stats = estadisticas.loc[grado]
        texto = (
            f"Muestra: {int(stats['count'])}\n"
            f"Media: {stats['mean']:.1f}%\n"
            f"Mediana: {stats['median']:.1f}%\n"
            f"Mín: {stats['min']:.1f}%\n"
            f"Máx: {stats['max']:.1f}%"
        )
        plt.annotate(
            texto,
            xy=(i, y_max * 0.9),
            ha='center',
            va='top',
            fontsize=9,
            bbox=dict(boxstyle="round,pad=0.4", facecolor="white", alpha=0.8)
        )

plt.show()


#### Distribución de partidas y errores por categoría en minijuego Vocabulario

In [ ]:
# Info por categoría
errores_por_categoria = {}
aciertos_por_categoria = {}
partidas_por_categoria = {}

tasa_errores_categoria = []

for jugador, fechas in partidas_usuarios.items():
    for partidas in fechas.values():
        for partida in partidas:
            if partida["gameType"] == "VOCABULARY":
                categoria = partida.get("category")
                errores = partida.get("incorrectas", 0)
                aciertos = partida.get("correctas", 0)
                total = errores + aciertos

                if categoria:
                    errores_por_categoria[categoria] = errores_por_categoria.get(categoria, 0) + errores
                    aciertos_por_categoria[categoria] = aciertos_por_categoria.get(categoria, 0) + aciertos
                    partidas_por_categoria[categoria] = partidas_por_categoria.get(categoria, 0) + 1

                    if total > 0:
                        tasa_error = (errores / total) * 100
                        tasa_errores_categoria.append({
                          'categoria': categoria,
                          'tasa_error': tasa_error
                        })

tasa_error_categoria = {}
for categoria in errores_por_categoria:
    total = errores_por_categoria[categoria] + aciertos_por_categoria.get(categoria, 0)
    if total > 0:
      tasa_error_categoria[categoria] = errores_por_categoria[categoria] / total
    else:
       tasa_error_categoria[categoria] = 0

categorias = list(errores_por_categoria.keys())
partidas = list(partidas_por_categoria.values())
tasas = [tasa_error_categoria[e] for e in categorias]

# Distribución de partidas por categoría
plt.figure(figsize=(10, 6))
plt.pie(partidas, labels=categorias, autopct=lambda pct: f'{pct:.0f}%\n({round(pct * sum(partidas) / 100)} partidas)')
plt.title('Distribución de partidas por categoría en el minijuego Vocabulario')
plt.tight_layout()
plt.show()

In [ ]:
# Convertimos a DataFrame
df_tasa_cat = pd.DataFrame(tasa_errores_categoria)

estadisticas = df_tasa_cat.groupby('categoria')['tasa_error'].agg(['count', 'mean', 'median', 'min', 'max'])
orden_categorias = estadisticas.index.tolist()

# Boxplot
plt.figure(figsize=(10, 6))
sns.boxplot(
    x='categoria',
    y='tasa_error',
    data=df_tasa_cat,
    medianprops=dict(color="red", linewidth=2.5),
    order=orden_categorias
)

# Añadir puntos individuales
sns.stripplot(
    x='categoria',
    y='tasa_error',
    data=df_tasa_cat,
    size=4,
    color='.3',
    alpha=0.5,
    order=orden_categorias
)

plt.title("Distribución de tasas de error por categoría de vocabulario")
plt.xlabel("Categoría")
plt.ylabel("Tasa de error (%)")
plt.ylim(0, 100)
plt.xticks(rotation=45, ha='right')
plt.grid(axis='y', linestyle='--', alpha=0.6)

# Añadir la media como rombo rojo
for i, categoria in enumerate(orden_categorias):
    media = estadisticas.loc[categoria, 'mean']
    plt.plot(i, media, marker='D', color='red', markersize=8, label='Media' if i==0 else "")

# Añadir anotaciones por categoría
y_max = df_tasa_cat['tasa_error'].max() * 1.1

for i, categoria in enumerate(orden_categorias):
    if categoria in estadisticas.index:
        stats = estadisticas.loc[categoria]
        stats_text = (
            f"Muestra: {int(stats['count'])}\n"
            f"Media: {stats['mean']:.1f}%\n"
            f"Mediana: {stats['median']:.1f}%\n"
            f"Mín: {stats['min']:.1f}%\n"
            f"Máx: {stats['max']:.1f}%"
        )

        plt.annotate(
            stats_text,
            xy=(i, y_max),
            ha='center',
            va='top',
            bbox=dict(boxstyle="round,pad=0.4", facecolor="white", alpha=0.8),
            fontsize=9
        )

plt.tight_layout()
plt.show()

#### Distribución de partidas y errores por emoción en minijuego Emociones

In [ ]:
# Info por emoción
errores_por_emocion = {}
aciertos_por_emocion = {}
partidas_por_emocion = {}

tasa_errores_emocion = []

for jugador, fechas in partidas_usuarios.items():
    for partidas in fechas.values():
        for partida in partidas:
            if partida["gameType"] == "EMOTIONS":
                emocion = partida.get("emotion")
                errores = partida.get("incorrectas", 0)
                aciertos = partida.get("correctas", 0)
                total = errores + aciertos

                if emocion:
                    errores_por_emocion[emocion] = errores_por_emocion.get(emocion, 0) + errores
                    aciertos_por_emocion[emocion] = aciertos_por_emocion.get(emocion, 0) + aciertos
                    partidas_por_emocion[emocion] = partidas_por_emocion.get(emocion, 0) + 1

                    if total > 0:
                        tasa_error = (errores / total) * 100
                        tasa_errores_emocion.append({
                          'categoria': emocion,
                          'tasa_error': tasa_error
                        })

tasa_error_emocion = {}
for emocion in errores_por_emocion:
    total = errores_por_emocion[emocion] + aciertos_por_emocion[emocion]
    if total > 0:
        tasa_error_emocion[emocion] = errores_por_emocion[emocion] / total
    else:
        tasa_error_emocion[emocion] = 0

emociones = list(errores_por_emocion.keys())
partidas = list(partidas_por_emocion.values())
tasas = [tasa_error_emocion[e] for e in emociones]

# Distribución de partidas por emoción
plt.figure(figsize=(10, 6))
plt.pie(partidas, labels=emociones, autopct=lambda pct: f'{pct:.0f}%\n({round(pct * sum(partidas) / 100)} partidas)')
plt.title('Distribución de partidas por emoción en el minijuego Emociones')
plt.tight_layout()
plt.show()

In [ ]:
# Convertimos a DataFrame
df_tasa_cat = pd.DataFrame(tasa_errores_emocion)

estadisticas = df_tasa_cat.groupby('categoria')['tasa_error'].agg(['count', 'mean', 'median', 'min', 'max'])
orden_categorias = estadisticas.index.tolist()

# Boxplot
plt.figure(figsize=(10, 6))
sns.boxplot(
    x='categoria',
    y='tasa_error',
    data=df_tasa_cat,
    medianprops=dict(color="red", linewidth=2.5),
    order=orden_categorias
)

# Añadir puntos individuales
sns.stripplot(
    x='categoria',
    y='tasa_error',
    data=df_tasa_cat,
    size=4,
    color='.3',
    alpha=0.5,
    order=orden_categorias
)

plt.title("Distribución de tasas de error por emoción de Emociones")
plt.xlabel("Categoría")
plt.ylabel("Tasa de error (%)")
plt.ylim(0, 100)
plt.xticks(rotation=45, ha='right')
plt.grid(axis='y', linestyle='--', alpha=0.6)

# Añadir la media como rombo rojo
for i, categoria in enumerate(orden_categorias):
    media = estadisticas.loc[categoria, 'mean']
    plt.plot(i, media, marker='D', color='red', markersize=8, label='Media' if i==0 else "")

# Añadir anotaciones por categoría
y_max = df_tasa_cat['tasa_error'].max() * 1.55

for i, categoria in enumerate(orden_categorias):
    if categoria in estadisticas.index:
        stats = estadisticas.loc[categoria]
        stats_text = (
            f"Muestra: {int(stats['count'])}\n"
            f"Media: {stats['mean']:.1f}%\n"
            f"Mediana: {stats['median']:.1f}%\n"
            f"Mín: {stats['min']:.1f}%\n"
            f"Máx: {stats['max']:.1f}%"
        )

        plt.annotate(
            stats_text,
            xy=(i, y_max),
            ha='center',
            va='top',
            bbox=dict(boxstyle="round,pad=0.4", facecolor="white", alpha=0.8),
            fontsize=9
        )

plt.tight_layout()
plt.show()

#### Distribución de partidas y errores por secuencia en minijuego Secuencia

In [ ]:
# Info por secuencia
errores_por_secuencia = {}
partidas_por_secuencia = {}
aciertos_por_secuencia = {}

tasa_errores_secuencia = []

for jugador, fechas in partidas_usuarios.items():
    for partidas in fechas.values():
        for partida in partidas:
            if partida["gameType"] == "SEQUENCE":
                secuencia = partida.get("sequence")
                errores = partida.get("incorrectas", 0)
                aciertos = partida.get("correctas", 0)
                total = errores + aciertos

                if secuencia:
                    errores_por_secuencia[secuencia] = errores_por_secuencia.get(secuencia, 0) + errores
                    aciertos_por_secuencia[secuencia] = aciertos_por_secuencia.get(secuencia, 0) + aciertos
                    partidas_por_secuencia[secuencia] = partidas_por_secuencia.get(secuencia, 0) + 1

                    if total > 0:
                        tasa_error = (errores / total) * 100
                        tasa_errores_secuencia.append({
                          'categoria': secuencia,
                          'tasa_error': tasa_error
                        })

tasa_error_secuencia = {}
for secuencia in errores_por_secuencia:
    total = errores_por_secuencia[secuencia] + aciertos_por_secuencia.get(secuencia, 0)
    if total > 0:
      tasa_error_secuencia[secuencia] = errores_por_secuencia[secuencia] / total
    else:
       tasa_error_secuencia[secuencia] = 0

# Convertimos a DataFrame
secuencias = list(errores_por_secuencia.keys())
partidas = list(partidas_por_secuencia.values())
tasas = [tasa_error_secuencia[e] for e in secuencias]

# Distribución de errores por secuencia en minijuego Secuencia
plt.figure(figsize=(10, 6))
plt.pie(partidas, labels=secuencias, autopct=lambda pct: f'{pct:.0f}%\n({round(pct * sum(partidas) / 100)} partidas)')
plt.title('Distribución de partidas por secuencia en el minijuego Secuencia')
plt.tight_layout()
plt.show()

# Análisis preguntas cuestionario en escala Likert 

In [ ]:
# Convertimos a DataFrame
df_tasa_cat = pd.DataFrame(tasa_errores_secuencia)

estadisticas = df_tasa_cat.groupby('categoria')['tasa_error'].agg(['count', 'mean', 'median', 'min', 'max'])
orden_categorias = estadisticas.index.tolist()

# Boxplot
plt.figure(figsize=(10, 6))
sns.boxplot(
    x='categoria',
    y='tasa_error',
    data=df_tasa_cat,
    medianprops=dict(color="red", linewidth=2.5),
    order=orden_categorias
)

# Añadir puntos individuales
sns.stripplot(
    x='categoria',
    y='tasa_error',
    data=df_tasa_cat,
    size=4,
    color='.3',
    alpha=0.5,
    order=orden_categorias
)

plt.title("Distribución de tasas de error por emoción de Emociones")
plt.xlabel("Categoría")
plt.ylabel("Tasa de error (%)")
plt.ylim(0, 100)
plt.xticks(rotation=45, ha='right')
plt.grid(axis='y', linestyle='--', alpha=0.6)

# Añadir la media como rombo rojo
for i, categoria in enumerate(orden_categorias):
    media = estadisticas.loc[categoria, 'mean']
    plt.plot(i, media, marker='D', color='red', markersize=8, label='Media' if i==0 else "")

# Añadir anotaciones por categoría
y_max = df_tasa_cat['tasa_error'].max() * 1.55

for i, categoria in enumerate(orden_categorias):
    if categoria in estadisticas.index:
        stats = estadisticas.loc[categoria]
        stats_text = (
            f"Muestra: {int(stats['count'])}\n"
            f"Media: {stats['mean']:.1f}%\n"
            f"Mediana: {stats['median']:.1f}%\n"
            f"Mín: {stats['min']:.1f}%\n"
            f"Máx: {stats['max']:.1f}%"
        )

        plt.annotate(
            stats_text,
            xy=(i, y_max),
            ha='center',
            va='top',
            bbox=dict(boxstyle="round,pad=0.4", facecolor="white", alpha=0.8),
            fontsize=9
        )

plt.tight_layout()
plt.show()

In [ ]:
# Datos por pregunta (ejemplo)
data = {
    "P1": [2, 2, 2, 2],
    "P2": [2, 4, 3, 2],
    "P3": [2, 5, 3, 5],
    "P4": [3, 2, 5, 3],
}

# Preguntas (ejemplo)
labels = {
    "P1": "P1 - Me gustaría usar este juego con frecuencia.",
    "P2": "P2 - El juego era innecesariamente complicado.",
    "P3": "P3 - El juego era sencillo de usar.",
    "P4": "P4 - Necesitaría soporte técnico para poder usar el \njuego."
}

# Convertimos a DataFrame
df_likert = pd.DataFrame(data)
df_T = df_likert.T
df_T["Pregunta"] = df_T.index.map(labels)
df = df_T.melt(id_vars="Pregunta", var_name="Usuario", value_name="Respuesta")


plt.figure(figsize=(10, 10))
sns.barplot(
    data=df,
    y="Pregunta", x="Respuesta", hue="Usuario",
    orient="h", palette="muted"
)

plt.title("Respuestas por pregunta (escala Likert 1-5)", fontsize=14)
plt.xlabel("Puntuación Likert (1-5)")
plt.ylabel("")
plt.xlim(0, 5.5)
plt.grid(axis='x', linestyle='--', alpha=0.5)
plt.legend().remove()
plt.tight_layout()
plt.show()
